In [1]:
#=======final code yolov5 in tikinter such that it can take photo , caputer video , live streammin and play recorded video 


import tkinter as tk
import cv2
from PIL import Image, ImageTk
import os
import torch
import numpy as np
from tkinter import messagebox
from tkVideoPlayer import TkinterVideo  # Import the TkinterVideo module

# IP camera credentials
username = 'admin'
password = 'Admin1234'
ip_address = '192.168.1.64'
stream_url = f"rtsp://{username}:{password}@{ip_address}/1"

# Function to capture and display a photo
def take_photo():
    capture = cv2.VideoCapture(stream_url)
    ret, frame = capture.read()
    if ret:
        cv2.imwrite("photo_from_ip_camera.jpg", frame)
        show_saved_photo()
    capture.release()


def live_video():
   
    capture = cv2.VideoCapture(stream_url)
    
    # Set the desired window size
    cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('frame', 900, 600)
    
    while True:
        ret, frame = capture.read()
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    capture.release()
    cv2.destroyAllWindows()

def yolov5_cctv_integration(username, password, ip_address, model_size='yolov5m', confidence_threshold=0.25):
    # Build the RTSP stream URL
    stream_url = f"rtsp://{username}:{password}@{ip_address}/1"

    # Load YOLOv5 model
    model = torch.hub.load('ultralytics/yolov5', model_size)
    model.eval()

    # Initialize the video capture
    cap = cv2.VideoCapture(stream_url)

    while True:
        ret, frame = cap.read()  # Read a frame from the camera

        # Check if the frame is valid
        if not ret:
            print("Error: Unable to read frame from the camera.")
            break

        # Perform inference
        results = model(frame)

        # Get detected objects and their details
        detections = results.pred[0]
        for det in detections:
            label = model.names[int(det[-1])]
            confidence = det[4].item()
            bbox = det[:4].tolist()

            if label in ['person', 'truck']:  # Include both "person" and "truck" labels
                confidence = det[4].item()
                bbox = det[:4].tolist()
                # Draw bounding boxes and labels
                
                if confidence > confidence_threshold:
                    x1, y1, x2, y2 = map(int, bbox)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f'{label}: {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    
                    if label == 'person':
                        print (" off weighting machine ")


        
        # Display the frame
        cv2.imshow('YOLOv5 Object Detection', frame)

        # Break the loop when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture and close OpenCV windows
    cap.release()
    cv2.destroyAllWindows()


# Function to perform YOLOv5 object detection on live video
def start_yolov5_detection():
    yolov5_cctv_integration(username, password, ip_address, model_size='yolov5s', confidence_threshold=0.25)

# Function to play recorded video
def play_recorded_video():
    recorded_video_path = "sample.mp4"  # Replace with your recorded video path
    video_player = TkinterVideo(master=window, scaled=True)
    video_player.load(r"{}".format(recorded_video_path))
    video_player.pack(expand=True, fill="both")
    video_player.play()

# Function to display the saved photo
def show_saved_photo():
    # Specify the destination folder
    destination_folder = r"C:\Users\User\Desktop\suraj\yolov5-master\yolov5-master\cammera-pic"
    
    # Load the saved image
    saved_image = Image.open("photo_from_ip_camera.jpg")
    
    try:
        # Use 'ANTIALIAS' for Pillow versions prior to 8.0.0
        saved_image = saved_image.resize((400, 300), Image.ANTIALIAS)
    except AttributeError:
        # Use 'antialias' for Pillow versions 8.0.0 and above
        saved_image = saved_image.resize((400, 300), Image.ANTIALIAS.lower())
    
    # Save the image to the destination folder
    saved_image_path = os.path.join(destination_folder, "saved_photo.jpg")
    saved_image.save(saved_image_path)
    
    # Create a PhotoImage object from the saved image
    saved_photo = ImageTk.PhotoImage(saved_image)
    
    # Update the label with the saved image
    saved_label.config(image=saved_photo)
    saved_label.image = saved_photo
    saved_label.pack(pady=10)

# Function to close the application
def close_app():
    if messagebox.askokcancel("Close", "Do you want to close the application?"):
        window.destroy()

# Create the main Tkinter window
window = tk.Tk()
window.title("Capture Photo from IP Camera")

# Set the custom window size (width x height)
window.geometry("600x500")

# Create a button to take a photo
photo_button = tk.Button(window, text="Take Photo", command=take_photo)
photo_button.pack(pady=10)

close_button = tk.Button(window, text="Live Video", command=live_video)
close_button.pack(pady=10)

# Create a button to start YOLOv5 object detection on live video
yolov5_button = tk.Button(window, text="Start YOLOv5 Detection", command=start_yolov5_detection)
yolov5_button.pack(pady=10)

# Create a button to play recorded video
play_video_button = tk.Button(window, text="Play Recorded Video", command=play_recorded_video)
play_video_button.pack(pady=10)

# Create a label to display the saved photo
saved_label = tk.Label(window)   

close_button = tk.Button(window, text="Close", command=close_app)
close_button.pack(pady=10)

# Run the Tkinter main loop
window.mainloop()


ModuleNotFoundError: No module named 'tkVideoPlayer'